The data for this file has been taken from cornell bird classification competition of kaggle. The entire dataset was around 22 gb.As this notebook has been made in colab so there is space restrictions. As such, I have taken a few audio files from the dataset and only two target classes of the birds for training the CNN

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Recursive copy
 
#!cp  "/content/kaggle" -r  "/content/drive/My Drive/ComputerVision/BirdCall"

In [2]:
!unzip -q '/content/drive/My Drive/ComputerVision/BirdCall/train.csv.zip'

In [3]:
path='/content/train.csv'

In [35]:
import pandas as pd
import matplotlib.pyplot as plt
import os

In [6]:
train_ds=pd.read_csv(path)

In [ ]:
train_ds.info()

In [ ]:
train_ds.head()

In [ ]:
train_ds.ebird_code.nunique()

264

In [ ]:
train_ds.shape

(21375, 35)

In [36]:
import IPython.display as ipd
import torch.nn as nn
import os
import numpy as np

In [8]:
!pip install torchaudio
import torch
import torchaudio

     |████████████████████████████████| 6.7MB 6.2MB/s 


In [24]:
from torchaudio.transforms import Spectrogram as sp
import torch.nn.functional as f
import torch.optim as optim
import torchvision.datasets as ds
from torch.utils.data import DataLoader as dl
import torchvision.transforms as tf
import shutil

In [10]:
!mkdir Spectro

In [ ]:
 
spect_out_path='/content/Spectro'
path='/content/drive/My Drive/ComputerVision/BirdCall/kaggle'
WF=[]
SR=[]
for f in os.listdir(path):
     #print(f[-3:])
     if f[-3:]=='mp3':
        filePath=os.path.join(path,f)
        wf, sr=torchaudio.load(filePath)
        WF.append(wf)
        SR.append(sr)
        #print(wf)
        plt.figure(figsize=(12,10))
        plt.plot(wf.t())
        plt.show()
        spg=sp()(wf)
        #print(spg.size())
        plt.figure(figsize=(8,8))
        plt.imshow(spg.log2()[0,:,:].numpy())
        plt.savefig(spect_out_path+'/%s.jpg'% f.split('.')[0])

In [31]:
class NN(nn.Module):
    def __init__(self):
       super(NN,self).__init__()
       self.conv1=nn.Conv2d(3,8,3,padding=1)
       self.conv2=nn.Conv2d(8,16,3,padding=1)
       self.conv3=nn.Conv2d(16,32,3,padding=1)
       self.pool=nn.MaxPool2d(2,2)
       self.linear1=nn.Linear(32*32*32, 128)
       self.linear2=nn.Linear(128, 64)
       self.linear3=nn.Linear(64, 2)
 
    def forward(self, inp):
       for conv in [self.conv1, self.conv2, self.conv3]:
          inp=self.pool(f.relu(conv(inp)))
       print(inp.shape)
       inp=inp.view(-1, inp.shape[1]*inp.shape[2]*inp.shape[3])
       print(inp.shape)
       inp=f.relu(self.linear1(inp))
       inp=f.relu(self.linear2(inp))
       inp=self.linear3(inp)
       return inp

In [13]:
dev='cuda' if torch.cuda.is_available() else 'cpu'

In [32]:
m=NN().to(dev)

In [33]:
criterion=nn.CrossEntropyLoss()
opt=optim.Adam(m.parameters(),lr=.01)

Create the file structure needed for ImageFolder to read the images fikes and classes.

In [16]:
files=[f[0:-3]+'mp3' for f in os.listdir(spect_out_path)]
files
T=pd.DataFrame(files, columns=['filename'])
 
d=pd.merge(train_ds,T,left_on='filename',right_on='filename', how='inner')
list(d.ebird_code.unique())

['amebit', 'balori']

In [17]:
def restructure_dir(df,path):
    files=os.listdir(path)
    for target in list(df.ebird_code.unique()):
 
        tpath=os.path.join(path,target)
        #print(tpath)
        if not os.path.exists(tpath):
           os.mkdir(tpath)
        tpath=''
    for f in files:
       if f.endswith('.jpg'):
          #print('files:',f[:-2])
          Class=df[df['filename'].str.contains(f[:-3])].ebird_code.item()
          #print(Class)
          src=os.path.join(path,f)
          #print(src)
          dst=os.path.join(path,str(Class))
          #print(dst)
          shutil.move(src,dst)

In [ ]:
#!rm -rf '/content/Spectro/balo

In [18]:
 restructure_dir(d,spect_out_path)

In [19]:
trans=tf.Compose([tf.RandomResizedCrop(256),tf.ToTensor()])

In [20]:
 
tr_img=ds.ImageFolder('/content/Spectro',transform=trans)
 
tr_ldr=dl(tr_img, batch_size=5, shuffle=True)

In [21]:
for data, lbl in tr_ldr:
    print(data.shape)
    print(lbl)

torch.Size([5, 3, 256, 256])
tensor([0, 1, 1, 0, 0])
torch.Size([5, 3, 256, 256])
tensor([0, 1, 1, 1, 1])
torch.Size([5, 3, 256, 256])
tensor([1, 0, 0, 0, 1])
torch.Size([5, 3, 256, 256])
tensor([1, 0, 0, 1, 0])
torch.Size([5, 3, 256, 256])
tensor([0, 0, 1, 1, 1])


In [22]:
def nn_train(model, d_ldr, epochs, criterion,opt):
        tr_loss=[]
        model.train()
        for e in range(epochs):
          print( 'epoch:',e)
          for data, lbl in d_ldr:
                 data=data.to(dev)
                 lbl=lbl.to(dev)
                 opt.zero_grad()
                 out=model(data)
                 #print(out.shape)
                 #print(lbl.long().shape)
                 loss=criterion(out, lbl)
                 loss.backward()
                 opt.step()
                 del out, data, lbl
                 torch.cuda.empty_cache()
                 tr_loss.append(loss.item())
        cp={'state_dict': model.state_dict()}
        torch.save(cp, 'Checkpoint.pth')     
        return tr_loss, model
 
def nn_eval(model, tst_ldr,epoch, criterion):
         tst_loss=[]
         true=0
         total=0
         model.eval()
         for data, lbl in tst_ldr:
            data=data.to(dev)
            lbl=lbl.to(dev)
            out=model(data)
            loss =criterion(out, lbl.long())
            _, pred=torch.max(out.data,1)
            total+=lbl.size(0)
            true+=(pred==lbl).sum().item()
            acc=100*true/total
         return tst_loss,acc

In [34]:
tr_loss, model=nn_train(m,tr_ldr,20, criterion, opt)

epoch: 0
torch.Size([5, 32, 32, 32])
torch.Size([5, 32768])
torch.Size([5, 32, 32, 32])
torch.Size([5, 32768])
torch.Size([5, 32, 32, 32])
torch.Size([5, 32768])
torch.Size([5, 32, 32, 32])
torch.Size([5, 32768])
torch.Size([5, 32, 32, 32])
torch.Size([5, 32768])
epoch: 1
torch.Size([5, 32, 32, 32])
torch.Size([5, 32768])
torch.Size([5, 32, 32, 32])
torch.Size([5, 32768])
torch.Size([5, 32, 32, 32])
torch.Size([5, 32768])
torch.Size([5, 32, 32, 32])
torch.Size([5, 32768])
torch.Size([5, 32, 32, 32])
torch.Size([5, 32768])
epoch: 2
torch.Size([5, 32, 32, 32])
torch.Size([5, 32768])
torch.Size([5, 32, 32, 32])
torch.Size([5, 32768])
torch.Size([5, 32, 32, 32])
torch.Size([5, 32768])
torch.Size([5, 32, 32, 32])
torch.Size([5, 32768])
torch.Size([5, 32, 32, 32])
torch.Size([5, 32768])
epoch: 3
torch.Size([5, 32, 32, 32])
torch.Size([5, 32768])
torch.Size([5, 32, 32, 32])
torch.Size([5, 32768])
torch.Size([5, 32, 32, 32])
torch.Size([5, 32768])
torch.Size([5, 32, 32, 32])
torch.Size([5, 327